In [1]:
# !pip install ydata-profiling pandas

In [2]:
import os
import logging
import warnings

# отключаем ворнинги
warnings.filterwarnings("ignore")

# отключаем прогресс-бары
os.environ["YDATA_PROFILING_DISABLE_PROGRESS_BAR"] = "1"
os.environ["PROGRESS_BAR_OFF"] = "True"

# отключаем логи библиотек
logging.getLogger("ydata_profiling").setLevel(logging.CRITICAL)
logging.getLogger("pandas_profiling").setLevel(logging.CRITICAL)
logging.getLogger("matplotlib").setLevel(logging.CRITICAL)

%matplotlib inline

In [3]:
import sqlite3
from pathlib import Path

import pandas as pd

try:
    from ydata_profiling import ProfileReport
    HAS_PROFILING = True
except ImportError:
    HAS_PROFILING = False
    print("⚠ ydata_profiling не установлен — профили не будут строиться.")


from pathlib import Path

ROOT = Path.cwd()   # текущая директория ноутбука
DB_PATH = ROOT.parent / "db" / "shop.db"
print("Используем БД:", DB_PATH.resolve())

conn = sqlite3.connect(DB_PATH)


Используем БД: C:\code\Projects\RecSys_case_final_arc\db\shop.db


In [4]:
def list_tables(connection):
    query = """
    SELECT name
    FROM sqlite_master
    WHERE type='table'
      AND name NOT LIKE 'sqlite_%'
    ORDER BY name
    """
    return pd.read_sql(query, connection)["name"].tolist()

tables = list_tables(conn)
tables


['category',
 'event',
 'order_item',
 'orders',
 'product',
 'recipe',
 'recipe_product',
 'session_info',
 'user']

In [5]:
def load_table(connection, table_name: str) -> pd.DataFrame:
    return pd.read_sql(f"SELECT * FROM {table_name}", connection)

# Пример: посмотрим на product
if "product" in tables:
    df_product = load_table(conn, "product")
    display(df_product.head())


,id,name,image_url,price,unit,weight_value,weight_unit,rating,shelf_life_days,category_id,brand,description,composition,metadata
0,1,"Абрикос мытый, Фрешбар",https://img.vkusvill.ru/pim/images/site/62d3b0...,165.0,шт,300.0,г,0.0,None,0,ВкусВилл,"Медовой спелости абрикосы, тщательно отобранны...",абрикос свежий,"{""category"": ""Фрукты"", ""subcategory"": ""Фруктов..."
1,2,Абрикос Планета Витаминов без косточки заморож...,https://img.vkusvill.ru/pim/images/site/a2549a...,199.0,шт,250.0,г,4.6,None,1,Планета Витаминов,Линейка Гурманики дополняется новым продуктом ...,Абрикос,"{""category"": ""Замороженные ягоды"", ""subcategor..."
2,3,Абрикосы сушеные,https://img.vkusvill.ru/pim/images/site/96ffdd...,328.0,шт,200.0,г,4.8,None,2,ВкусВилл,Сушеные абрикосы способны быстро утолить голод...,абрикосы сушеные без косточки,"{""category"": ""Орехи, чипсы и снеки"", ""subcateg..."
3,5,Аджика,https://img.vkusvill.ru/pim/images/site/site/2...,184.0,шт,200.0,г,4.8,None,3,ВкусВилл,Острая аджика с насыщенным вкусом и густой тек...,"перец красный сладкий свежий, паста томатная (...","{""category"": ""Консервация"", ""subcategory"": ""Ху..."
4,6,Аджика / Соус Фирменный KERAKUR,https://img.vkusvill.ru/pim/images/site/259f77...,335.0,шт,480.0,г,4.8,None,1,Kerakur,"Аджика неострая , подается к мясным и куриным ...","красный перец, томатная паста, красный острый ...","{""category"": ""Овощные консервы"", ""subcategory""..."


In [6]:
reports_dir = Path("table_reports")
reports_dir.mkdir(exist_ok=True)

key_tables = ["user", "product", "orders", "order_item", "event"]

for t in key_tables:
    if t not in tables:
        continue
    print("=" * 60)
    print(f"⭐ Ключевая таблица: {t}")
    print("=" * 60)
    
    df = load_table(conn, t)
    display(df)
    
    if HAS_PROFILING and len(df) > 0:
        profile = ProfileReport(df, title=f"Key table profile: {t}", minimal=True)
        html_path = reports_dir / f"key_{t}_profile.html"
        profile.to_file(html_path)
        print(f"Profile для {t} сохранён в {html_path}\n")


⭐ Ключевая таблица: user


,id,username,password_hash,created_at
0,1,asnp,a665a45920422f9d417e4867efdc4fb8a04a1f3fff1fa0...,2025-11-15 09:36:46
1,2,test,a665a45920422f9d417e4867efdc4fb8a04a1f3fff1fa0...,2025-11-15 09:49:22
2,3,new_akk,a665a45920422f9d417e4867efdc4fb8a04a1f3fff1fa0...,2025-11-15 11:00:48
3,4,test2,a665a45920422f9d417e4867efdc4fb8a04a1f3fff1fa0...,2025-11-15 13:49:29
4,5,test3,a665a45920422f9d417e4867efdc4fb8a04a1f3fff1fa0...,2025-11-15 13:50:27
5,6,test_10,a665a45920422f9d417e4867efdc4fb8a04a1f3fff1fa0...,2025-11-16 21:45:32


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<00:00, 879.26it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

...

100%|██████████| 14/14 [00:00<00:00, 260.92it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Profile для event сохранён в table_reports\key_event_profile.html



In [7]:
def max_matching(adj: dict[int, list[int]]) -> dict[int, int]:

    matchR = {}

    def dfs(u, seen):
        for v in adj[u]:
            if v in seen:
                continue
            seen.add(v)

            if v not in matchR or dfs(matchR[v], seen):
                matchR[v] = u
                return True
        return False

    for u in adj:
        dfs(u, set())

    matchL = {u: v for v, u in matchR.items()}
    return matchL


In [8]:
adj = {
    1:  [2, 5, 7],
    2:  [1, 3],
    3:  [4, 8, 9, 12],
    4:  [5],
    5:  [1, 6, 7, 8],
    6:  [3, 9],
    7:  [2, 4, 6],
    8:  [10, 11],
    9:  [5, 7, 12],
    10: [8]
}



matching = max_matching(adj)
print(len(matching))
print(matching)


10
{7: 2, 2: 1, 3: 4, 4: 5, 6: 3, 5: 6, 1: 7, 8: 10, 10: 8, 9: 12}
